In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy
from decimal import *
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr

In [2]:
import threading
import numpy as np
import datetime

In [3]:
import scipy.stats as ss

In [4]:
import itertools

In [5]:
def setupStake(
        # Number of nodes in the network. This number can be changed to test with small and large network size
        numNodes=5,
        # the minimum number of channels a node can open
        minChannelsPerNode=2,
        # the maximum number of channels a node can open 
        maxChannelsPerNode=10,
        minFundsPerNode=10,
        maxFundsPerNode=100,
        tokensPerTicket=0.1
    ):

    stake = [[0 for i in range(numNodes)] for j in range(numNodes)]
    for x in range(numNodes):
        # each node is given a random funding amount
        myFunds = numpy.random.rand() * (maxFundsPerNode - minFundsPerNode) + minFundsPerNode

        # get random number of channels per node
        myChannels = int(numpy.random.rand() * (maxChannelsPerNode - minChannelsPerNode + 1) + minChannelsPerNode)

        # This value represents the amount each node stakes in their channel
        # It is computed as the number of funds a node has divided by number of channels they open
        stakePerChannel = myFunds / myChannels
        stakePerChannel = int(stakePerChannel / tokensPerTicket) * tokensPerTicket

        # fund channels by writing into stake matrix
        for c in range(myChannels):
            # TODO: this does not prevent a node from opening a channel to the same counterparty multiple times
            counterparty = int(numpy.random.rand() * (numNodes - 1))

            # cannot open channel to self - keep diagonal of matrix at 0
            if counterparty >= x:
                counterparty = counterparty + 1
            stake[x][counterparty] = stakePerChannel
            stake = [[Decimal(i) for i in j] for j in stake]

    return stake

In [6]:
initial_stake = setupStake()
initial_stake

[[Decimal('0'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('0'),
  Decimal('7.60000000000000053290705182007513940334320068359375')],
 [Decimal('0'),
  Decimal('0'),
  Decimal('4.800000000000000710542735760100185871124267578125'),
  Decimal('0'),
  Decimal('4.800000000000000710542735760100185871124267578125')],
 [Decimal('12.2000000000000010658141036401502788066864013671875'),
  Decimal('12.2000000000000010658141036401502788066864013671875'),
  Decimal('0'),
  Decimal('0'),
  Decimal('0')],
 [Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('3.600000000000000088817841970012523233890533447265625')],
 [Decimal('12.4000000000000003552713678800500929355621337890625'),
  Decimal('12.4000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('12.400

In [7]:
def calcImportance(stake):

    # get total stake per node
    stakePerNode = [Decimal(sum(e)) for e in stake]

    totalStake = sum(stakePerNode)

    n = len(stake)
    weight = [0] * n
    importance = [0] * n
    for x in range(n):
        for y in range(n):
            # weight of a node is 0 if its stake is equal to 0
            #weight(channel) = sqrt(balance(channel) / stake(channel.source) * stake(channel.destination))
            weight[y] += 0 if stakePerNode[y]==0 else numpy.sqrt(stake[y][x]/stakePerNode[y] * stakePerNode[x])

    #print("Weighted downstream stake per node p(n) = ", weight)
    importanceList = numpy.array(weight) * numpy.array(stakePerNode)
    return importanceList;

In [8]:
initial_importance = calcImportance(initial_stake)
initial_importance

array([Decimal('186.0962836928318760073595221'),
       Decimal('74.93396707088795936667244081'),
       Decimal('135.8416035312387964393975761'),
       Decimal('87.12389705129652691870752488'),
       Decimal('239.6807431009032342675378919')], dtype=object)

In [9]:
#MODEL1 = based on stake

In [10]:
numNodes=5
def recalculating_stakes(stake, importance):
    #calculating sum of the original stake
    sum_stake = sum([sum(stake[li]) for li in range(numNodes)])
    #creating an empy stake matrix with the original shape
    stake_mod = [[0 for i in range(numNodes)] for j in range(numNodes)]
    #populating it with recalculation
    for p in range(numNodes):
        for r in range(numNodes):
            if p == r:
                stake_mod[p][r] = 0
            else:
                stake_mod[p][r] = sum(stake[r])/(sum_stake-sum(stake[p]))*sum(stake[p])
    
    return stake_mod

In [11]:
based_on_stake = recalculating_stakes(initial_stake, initial_importance)
based_on_stake

[[0,
  Decimal('2.669268292682927252550562517'),
  Decimal('6.784390243902439688637802309'),
  Decimal('3.002926829268292788683993153'),
  Decimal('10.34341463414634186884879749')],
 [Decimal('2.299159663865546597780536353'),
  0,
  Decimal('2.460504201680672717242919264'),
  Decimal('1.089075630252100970505295689'),
  Decimal('3.751260504201681135556720215')],
 [Decimal('6.919402985074627580798056186'),
  Decimal('2.913432835820896050480600151'),
  0,
  Decimal('3.277611940298507652470648290'),
  Decimal('11.28955223880597084787890266')],
 [Decimal('2.619574468085106458029598833'),
  Decimal('1.102978723404255436682804729'),
  Decimal('2.803404255319149064825410021'),
  0,
  Decimal('4.274042553191489306915712328')],
 [Decimal('12.54674556213017774846334034'),
  Decimal('5.282840236686391042520063735'),
  Decimal('13.42721893491124308513698614'),
  Decimal('5.943195266272189189693713430'),
  0]]

In [58]:
pd.DataFrame(based_on_stake)

,0,1,2,3,4
0,0,2.669268292682927252550562517,6.784390243902439688637802309,3.002926829268292788683993153,10.34341463414634186884879749
1,2.299159663865546597780536353,0,2.460504201680672717242919264,1.089075630252100970505295689,3.751260504201681135556720215
2,6.919402985074627580798056186,2.913432835820896050480600151,0,3.277611940298507652470648290,11.28955223880597084787890266
3,2.619574468085106458029598833,1.102978723404255436682804729,2.803404255319149064825410021,0,4.274042553191489306915712328
4,12.54674556213017774846334034,5.282840236686391042520063735,13.42721893491124308513698614,5.943195266272189189693713430,0


In [59]:
def printArray2d(a, format=1):
    for row in range(len(a)):
        for col in range (len(a[row])):
            if format == 1:
                print("{:4.1f}".format(a[row][col]), end = " ")
            else:
                print("{:4.0f}".format(a[row][col]), end = " ")

        print()

In [61]:
printArray2d(initial_stake, format=1)

 0.0  7.6  7.6  0.0  7.6 
 0.0  0.0  4.8  0.0  4.8 
12.2 12.2  0.0  0.0  0.0 
 3.6  3.6  0.0  0.0  3.6 
12.4 12.4  0.0 12.4  0.0 


In [60]:
printArray2d(based_on_stake, format=1)

 0.0  2.7  6.8  3.0 10.3 
 2.3  0.0  2.5  1.1  3.8 
 6.9  2.9  0.0  3.3 11.3 
 2.6  1.1  2.8  0.0  4.3 
12.5  5.3 13.4  5.9  0.0 


In [12]:
calcImportance(based_on_stake)

array([Decimal('206.4627811495331197127163418'),
       Decimal('93.66766784755561033529925362'),
       Decimal('218.7851548894485915699244296'),
       Decimal('104.7098849201927127501761273'),
       Decimal('305.8554952914856700105630242')], dtype=object)

In [13]:
def iterating_recalculating_stakes(stake, importance):
    iterated_stakes = []
    iterated_importance = []
    for _ in range(100):
        a = recalculating_stakes(stake, importance)
        iterated_stakes.append(a)
        importance = calcImportance(a)
        iterated_importance.append(importance)
        stake = a 
    return iterated_stakes

In [45]:
iteration = iterating_recalculating_stakes(initial_stake, initial_importance)
iteration

[[[0,
   Decimal('2.669268292682927252550562517'),
   Decimal('6.784390243902439688637802309'),
   Decimal('3.002926829268292788683993153'),
   Decimal('10.34341463414634186884879749')],
  [Decimal('2.299159663865546597780536353'),
   0,
   Decimal('2.460504201680672717242919264'),
   Decimal('1.089075630252100970505295689'),
   Decimal('3.751260504201681135556720215')],
  [Decimal('6.919402985074627580798056186'),
   Decimal('2.913432835820896050480600151'),
   0,
   Decimal('3.277611940298507652470648290'),
   Decimal('11.28955223880597084787890266')],
  [Decimal('2.619574468085106458029598833'),
   Decimal('1.102978723404255436682804729'),
   Decimal('2.803404255319149064825410021'),
   0,
   Decimal('4.274042553191489306915712328')],
  [Decimal('12.54674556213017774846334034'),
   Decimal('5.282840236686391042520063735'),
   Decimal('13.42721893491124308513698614'),
   Decimal('5.943195266272189189693713430'),
   0]],
 [[0,
   Decimal('2.669268292682927252550562518'),
   Decimal('6

In [15]:
#MODEL2_based on stake and importance

In [16]:
numNodes=5
def recalculating_stakes_with_importance(stake, importance):
    #calculating sum of the original stake
    sum_stake = sum([sum(stake[li]) for li in range(numNodes)])
    #creating an empy stake matrix with the original shape
    stake_mod = [[0 for i in range(numNodes)] for j in range(numNodes)]
    stake_mod_i = [[0 for i in range(numNodes)] for j in range(numNodes)]
    #populating it with recalculation
    for p in range(numNodes):
        for r in range(numNodes):
            if stake[p][r] == 0:
                stake_mod[p][r] = 0
            else:
                stake_mod[p][r] = sum(stake[r])/(sum_stake-sum(stake[p]))*sum(stake[p])
                
    for y in range(numNodes):
        for z in range(numNodes):
            if y == z:
                stake_mod_i[y][z] = 0
            else:
                stake_mod_i[y][z] = importance[z]/(sum(importance)-importance[y])*sum(stake_mod[y])
    
    return stake_mod_i

In [17]:
recalculating_stakes_with_importance(initial_stake, initial_importance)

[[0,
  Decimal('2.759538389618130418870101416'),
  Decimal('5.002539362384476985457539477'),
  Decimal('3.208448023828102762653040551'),
  Decimal('8.826547394900998643056480878')],
 [Decimal('1.781887695283665635195549323'),
  0,
  Decimal('1.300694871690444704729543655'),
  Decimal('0.8342187014175496068715390483'),
  Decimal('2.294963437490693906003007453')],
 [Decimal('3.112871033529964827722444353'),
  Decimal('1.253435968165118913048563243'),
  0,
  Decimal('1.457339448577468098557291795'),
  Decimal('4.009189370622971791950356946')],
 [Decimal('2.337806422229664469531452026'),
  Decimal('0.9413466297404398936373728438'),
  Decimal('1.706489602261403217951738369'),
  0,
  Decimal('3.010953090449343620507552650')],
 [Decimal('9.140630257498580907596294130'),
  Decimal('3.680587667473897542007837014'),
  Decimal('6.672233570844783156450727574'),
  Decimal('4.279329569271496374622258792'),
  0]]

In [66]:
def iterating_recalculating_stakes_with_importance(stake, importance):
    iterated_stakes = []
    iterated_importance = []
    
    for _ in range(100):
        printArray2d(stake)
        print(" ")
        a = recalculating_stakes_with_importance(stake, importance)
        iterated_stakes.append(a)
        b = calcImportance(a)
        iterated_importance.append(importance)
        stake = a
        importance = b
        
    return iterated_stakes, iterated_importance

In [67]:
iterating_recalculating_stakes_with_importance(initial_stake, initial_importance)

 0.0  7.6  7.6  0.0  7.6 
 0.0  0.0  4.8  0.0  4.8 
12.2 12.2  0.0  0.0  0.0 
 3.6  3.6  0.0  0.0  3.6 
12.4 12.4  0.0 12.4  0.0 
 
 0.0  2.8  5.0  3.2  8.8 
 1.8  0.0  1.3  0.8  2.3 
 3.1  1.3  0.0  1.5  4.0 
 2.3  0.9  1.7  0.0  3.0 
 9.1  3.7  6.7  4.3  0.0 
 
 0.0  2.8  4.3  3.6  9.1 
 2.0  0.0  1.1  0.9  2.3 
 3.3  1.2  0.0  1.5  3.8 
 2.6  0.9  1.4  0.0  3.0 
10.1  3.6  5.5  4.6  0.0 
 
 0.0  2.8  4.3  3.6  9.1 
 2.0  0.0  1.1  0.9  2.3 
 3.3  1.2  0.0  1.5  3.8 
 2.6  0.9  1.4  0.0  3.0 
10.1  3.6  5.5  4.6  0.0 
 
 0.0  2.8  4.3  3.6  9.1 
 2.0  0.0  1.1  0.9  2.3 
 3.3  1.2  0.0  1.5  3.8 
 2.6  0.9  1.4  0.0  3.0 
10.1  3.6  5.5  4.6  0.0 
 
 0.0  2.8  4.3  3.6  9.1 
 2.0  0.0  1.1  0.9  2.3 
 3.3  1.2  0.0  1.5  3.8 
 2.6  0.9  1.4  0.0  3.0 
10.1  3.6  5.5  4.6  0.0 
 
 0.0  2.8  4.3  3.6  9.1 
 2.0  0.0  1.1  0.9  2.3 
 3.3  1.2  0.0  1.5  3.8 
 2.6  0.9  1.4  0.0  3.0 
10.1  3.6  5.5  4.6  0.0 
 
 0.0  2.8  4.3  3.6  9.1 
 2.0  0.0  1.1  0.9  2.3 
 3.3  1.2  0.0  1.5  3.8

([[[0,
    Decimal('2.759538389618130418870101416'),
    Decimal('5.002539362384476985457539477'),
    Decimal('3.208448023828102762653040551'),
    Decimal('8.826547394900998643056480878')],
   [Decimal('1.781887695283665635195549323'),
    0,
    Decimal('1.300694871690444704729543655'),
    Decimal('0.8342187014175496068715390483'),
    Decimal('2.294963437490693906003007453')],
   [Decimal('3.112871033529964827722444353'),
    Decimal('1.253435968165118913048563243'),
    0,
    Decimal('1.457339448577468098557291795'),
    Decimal('4.009189370622971791950356946')],
   [Decimal('2.337806422229664469531452026'),
    Decimal('0.9413466297404398936373728438'),
    Decimal('1.706489602261403217951738369'),
    0,
    Decimal('3.010953090449343620507552650')],
   [Decimal('9.140630257498580907596294130'),
    Decimal('3.680587667473897542007837014'),
    Decimal('6.672233570844783156450727574'),
    Decimal('4.279329569271496374622258792'),
    0]],
  [[0,
    Decimal('2.813184855122057

In [20]:
#MODEL3: limited number of nodes 

In [71]:
numNodes=5
def recalculating_stakes_limited(stake, importance):
    #calculating sum of the original stake
    sum_stake = sum([sum(stake[li]) for li in range(numNodes)])
    #creating an empy stake matrix with the original shape
    stake_mod = [[0 for i in range(numNodes)] for j in range(numNodes)]
    #populating it with recalculation
    for p in range(numNodes):
        for r in range(numNodes):
            if stake[p][r] == 0:
                stake_mod[p][r] = 0     
            else:
                stake_mod[p][r] = (sum(stake[r])/(sum_stake-sum(stake[p]))*sum(stake[p]))
        
    return stake_mod

In [ ]:
  sum_stake_mod = sum([sum(stake_mod[li]) for li in range(numNodes)])
     
    for h in range(numNodes):   
        for j in range(numNodes):
            if stake[h][j] == 0:
                stake_mod[h][j] = 0     
            else:
                stake_mod[h][j] = stake_mod[h][j]/sum(stake_mod[h])*sum(stake[h])*sum(stake[h])/sum(stake_mod[h])

In [73]:
limited2 = recalculating_stakes_limited(initial_stake, initial_importance)

In [76]:
sum(initial_stake[0])

Decimal('22.80000000000000159872115546')

In [75]:
sum(limited2[0])

Decimal('19.79707317073170881003716232')

In [52]:
initial_stake

[[Decimal('0'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('0'),
  Decimal('7.60000000000000053290705182007513940334320068359375')],
 [Decimal('0'),
  Decimal('0'),
  Decimal('4.800000000000000710542735760100185871124267578125'),
  Decimal('0'),
  Decimal('4.800000000000000710542735760100185871124267578125')],
 [Decimal('12.2000000000000010658141036401502788066864013671875'),
  Decimal('12.2000000000000010658141036401502788066864013671875'),
  Decimal('0'),
  Decimal('0'),
  Decimal('0')],
 [Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('3.600000000000000088817841970012523233890533447265625')],
 [Decimal('12.4000000000000003552713678800500929355621337890625'),
  Decimal('12.4000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('12.400

In [68]:
limited = recalculating_stakes_limited(initial_stake, initial_importance)

In [69]:
printArray2d(initial_stake)

 0.0  7.6  7.6  0.0  7.6 
 0.0  0.0  4.8  0.0  4.8 
12.2 12.2  0.0  0.0  0.0 
 3.6  3.6  0.0  0.0  3.6 
12.4 12.4  0.0 12.4  0.0 


In [70]:
printArray2d(limited)

 0.0  3.5  8.3  0.0 11.0 
 0.0  0.0  5.9  0.0  3.7 
42.6  0.8  0.0  0.0  0.0 
 4.8  1.2  0.0  0.0  4.7 
30.7  4.2  0.0  4.9  0.0 


In [56]:
def iterating_recalculating_stakes_limited(stake, importance):
    iterated_stakes = []
    iterated_importance = []
    for _ in range(100):
        a = recalculating_stakes_limited(stake, importance)
        iterated_stakes.append(a)
        importance = calcImportance(a)
        iterated_importance.append(importance)
        stake = a 
    return iterated_stakes

In [57]:
iteration = iterating_recalculating_stakes_limited(initial_stake, initial_importance)

DivisionByZero: [<class 'decimal.DivisionByZero'>]

In [22]:
limited = recalculating_stakes_limited(initial_stake, initial_importance)

In [23]:
k = []
for j in range(5):
    l = [i for i in range(5)]
    k.append(l)

In [24]:
o = []
for g in range(5):
    h = [g]*5
    o.append(h)

In [25]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [26]:
df_00 = pd.DataFrame(flatten(o))

In [27]:
df_0 = pd.DataFrame(flatten(k))
df_0

,0
0,0
1,1
2,2
3,3
4,4
5,0
6,1
7,2
8,3
9,4


In [28]:
df = pd.DataFrame(flatten(initial_stake))

In [29]:
df_1 = pd.concat([df_0, df], axis=1)
df_1

,0,0
0,0,0
1,1,7.60000000000000053290705182007513940334320068...
2,2,7.60000000000000053290705182007513940334320068...
3,3,0
4,4,7.60000000000000053290705182007513940334320068...
5,0,0
6,1,0
7,2,4.80000000000000071054273576010018587112426757...
8,3,0
9,4,4.80000000000000071054273576010018587112426757...


In [30]:
df_2 = pd.concat([df_00, df_1], axis=1)
df_2.head(30)

,0,0,0
0,0,0,0
1,0,1,7.60000000000000053290705182007513940334320068...
2,0,2,7.60000000000000053290705182007513940334320068...
3,0,3,0
4,0,4,7.60000000000000053290705182007513940334320068...
5,1,0,0
6,1,1,0
7,1,2,4.80000000000000071054273576010018587112426757...
8,1,3,0
9,1,4,4.80000000000000071054273576010018587112426757...


iteration_recalculating_stakes(initial_stake)

In [31]:
import networkx as nx

In [32]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.


In [33]:
df_2.columns=['source', 'target', 'weight']

In [34]:
df_2['weight2'] = df_2['weight'].apply(lambda x: float(x))
df_2

,source,target,weight,weight2
0,0,0,0,0.0
1,0,1,7.60000000000000053290705182007513940334320068...,7.6
2,0,2,7.60000000000000053290705182007513940334320068...,7.6
3,0,3,0,0.0
4,0,4,7.60000000000000053290705182007513940334320068...,7.6
5,1,0,0,0.0
6,1,1,0,0.0
7,1,2,4.80000000000000071054273576010018587112426757...,4.8
8,1,3,0,0.0
9,1,4,4.80000000000000071054273576010018587112426757...,4.8


In [35]:
df_3 = df_2[df_2['weight2'] > 0]

In [36]:
G = nx.from_pandas_edgelist(df_3, source='target', target='source', edge_attr='weight2')

In [37]:
from pyvis.network import Network

In [38]:
net = Network(notebook=True, height='700px', width='700', directed=True, bgcolor="#222222", font_color='white')

In [39]:
net.from_nx(G)

In [43]:
initial_stake

[[Decimal('0'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('0'),
  Decimal('7.60000000000000053290705182007513940334320068359375')],
 [Decimal('0'),
  Decimal('0'),
  Decimal('4.800000000000000710542735760100185871124267578125'),
  Decimal('0'),
  Decimal('4.800000000000000710542735760100185871124267578125')],
 [Decimal('12.2000000000000010658141036401502788066864013671875'),
  Decimal('12.2000000000000010658141036401502788066864013671875'),
  Decimal('0'),
  Decimal('0'),
  Decimal('0')],
 [Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('3.600000000000000088817841970012523233890533447265625')],
 [Decimal('12.4000000000000003552713678800500929355621337890625'),
  Decimal('12.4000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('12.400

In [40]:
net.show("example.html")

In [41]:
net.show("match.html")